# Developing of the pipeline

In [2]:
!pip install pytorch-lightning==0.9.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytorch_lightning-0.9.0-py3-none-any.whl (408 kB)
ERROR: pytorch-lightning-bolts 0.2.5 has requirement pytorch-lightning>=0.10.0, but you'll have pytorch-lightning 0.9.0 which is incompatible.
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.0.5
    Uninstalling pytorch-lightning-1.0.5:
      Successfully uninstalled pytorch-lightning-1.0.5


In [ ]:
#from data_loaders import get_data_loader
# from utils.visualisation import showInRow

import torch
from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
import pytorch_lightning as pl
# from pytorch_lightning import seed_everything
# import torchvision.models as models
# from torchvision import transforms
# #from pytorch_lightning.loggers import WandbLogger

# #seed_everything(123456)

In [ ]:
transform=transforms.Compose([#transforms.Grayscale(),
                              transforms.Resize((256,256)),
                              transforms.ToTensor(),
                              #transforms.Normalize((0.5,), (0.5,))
                             ])

In [5]:
class ResNet18_Rotation(pl.LightningModule):
    def __init__(self, num_classes, dataset_name, pretrained=True, transform=None, mode="pretrain", batch_size=16):
        # Load model
        super(ResNet18_Rotation, self).__init__()
        self.model = models.resnet18(pretrained=pretrained)
        self.mode = mode
        self.mode_outputs = {
            "pretrain": 4,
            "train": num_classes
        }
        self.loss = {
            "pretrain": nn.CrossEntropyLoss(),
            "train": nn.BCEWithLogitsLoss()
        }
        self.opt = {
            "pretrain": SGD(self.parameters(), lr=1e-4, momentum=0.9),
            "train": SGD(self.parameters(), lr=1e-4, momentum=0.9)
        }
        self.batch_size = batch_size
        self.dataset = dataset_name
        self.transform = transform
    
    
    def forward(self, x):
        x = self.model(x)
        return x
    
    def training_step(self, batch, batch_idx):
        if self.mode == "pretrain":
            images, labels = \
                batch['image'], batch['label']
        else:
            images, labels = \
                batch['image'], batch['one_hot_label']
        images = images.cuda()
        labels = labels.cuda()
        logits = self(images)
        loss = self.loss[self.mode](logits, labels)
        
        return loss
    
    def configure_optimizers(self):
        return self.opt[self.mode]
          
    
    
    def change_mode(self, mode):
        if mode in self.mode_outputs:
            self.mode = mode
            self.model.fc = nn.Linear(in_features=512, out_features=self.mode_outputs[self.mode], bias=True)
        else:
            print("Unknown mode, only next available: ", list(self.mode_outputs.keys()))
            
            
    def train_dataloader(self):
        if self.mode == "pretrain":
            train_raw = get_data_loader(self.dataset, part="train")
            train = RotationWrapper(train_raw, transform=self.transform)
        else:
            train = get_data_loader(self.dataset, part="train", transform=self.transform)    
        return DataLoader(train, batch_size=self.batch_size, shuffle=True, num_workers=1)
    
    
    def val_dataloader(self):
        if self.mode == "pretrain":
            train_raw = get_data_loader(self.dataset, part="val")
            train = RotationWrapper(train_raw, transform=self.transform)
        else:
            train = get_data_loader(self.dataset, part="val", transform=self.transform)    
        return DataLoader(train, batch_size=self.batch_size, shuffle=False, num_workers=1)

    
    def test_dataloader(self):
        if self.mode == "pretrain":
            test_raw = get_data_loader(self.dataset, part="test")
            test = RotationWrapper(test_raw, transform=self.transform)
        else:
            test = get_data_loader(self.dataset, part="test", transform=self.transform)    
        return DataLoader(test, batch_size=self.batch_size, shuffle=False, num_workers=1)
    

In [6]:
rnet = ResNet18_Rotation(num_classes=15, dataset_name="Chest14", pretrained=True, 
                         transform=transform, mode="pretrain", batch_size=16)
if torch.cuda.is_available():
    rnet = rnet.cuda()
trainer = pl.Trainer(gpus=1, deterministic=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [7]:
trainer.fit(rnet)

/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping validation loop
  warnings.warn(*args, **kwargs)
/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11 M  
/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint..


/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: RuntimeWarning: The metric you returned None must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)
/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: RuntimeWarning: Can save best model only with loss available, skipping.
  warnings.warn(*args, **kwargs)


1

In [10]:
from pl_bolts.models.self_supervised import MocoV2

In [13]:
class MyDataModule(pl.LightningDataModule):

    def __init__(self, dataset: str, transform, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.dataset = dataset
        self.transform = transform

    def setup(self, stage=None):
        self.train = get_data_loader(self.dataset, transform=transform, part="train")
        self.val = get_data_loader(self.dataset, transform=transform, part="val")
        self.test = get_data_loader(self.dataset, transform=transform, part="test")

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)

In [23]:
data_module = MyDataModule(dataset="RSNA", transform=transform, batch_size=16)
wandb_logger = WandbLogger(name='Adam-32-0.001',project='pytorchlightning')
trainer = pl.Trainer(gpus=1, deterministic=True, logger=wandb_logger)
m = MocoV2(base_encoder='resnet18', datamodule=data_module, batch_size=16, )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [24]:
trainer.fit(m)

wandb: Currently logged in as: genvekt (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name      | Type   | Params
-------------------------------------
0 | encoder_q | ResNet | 11 M  
1 | encoder_k | ResNet | 11 M  
/home/intern/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  return wrapped_fn


ValueError: too many values to unpack (expected 2)